In [ ]:
import math
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from string import ascii_uppercase
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from scipy.stats import norm
import itertools
import numpy as np
from collections.abc import Iterable

plt.rcParams.update({
    "figure.dpi": 300,
    "savefig.dpi": 300,
    "savefig.format": "pdf",
    "lines.linewidth": 2,
    "axes.labelsize": 16,
    "axes.titlesize": 18,
    'axes.labelweight': 'bold',
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "legend.fontsize": 14,
    "grid.alpha": 0.5,
    "font.family": "DejaVu Sans",  # 更稳定跨平台字体
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
    "text.usetex": False,
    "text.antialiased": True
})

# rps熵计算

In [ ]:
def compute_rps_entropy(evidence):
    """
    严格遵循Example 4.1计算方法的RPS熵

    参数:
        evidence: 单个RPS证据体（字典格式，键为元组，值为质量）

    返回:
        entropy: 计算得到的熵值
    """
    total_entropy = 0.0

    # 预计算F(i)值（根据Example 4.1的算法）
    def compute_F(i):
        return sum(math.perm(i, k) for k in range(i + 1))

    # 计算每个证据项的贡献
    for items, mass in evidence.items():
        if mass <= 0:
            continue  # 跳过零质量项

        i = len(items)  # 当前组合长度
        F_i = compute_F(i)

        # 特别注意：Example中F(1)-1=2-1=1，但按定义F(1)=1! =1 → 需要确认
        # 根据Example 4.1的实际计算，F(1)=2, F(2)=5, F(3)=16
        # 这表明原定义可能有调整，这里采用示例中的值

        log_arg = mass / (F_i - 1)
        term = mass * math.log(log_arg)  # term本身是负的
        total_entropy -= term  # 负负得正
        # print(f"组合 {items}: mass={mass}, log参数={log_arg:.4f}, term={term:.4f}, 贡献={-term:.4f}")
    # hypothesis_complexity = 1 - 1 / len(evidence)
    #     print(f"证据 {evidence}的总rps熵: total_entropy={total_entropy}")
    # 标准化到0-1范围（熵值越小质量越高）
    max_possible_entropy = math.log(len(evidence) * 10)  # 经验值
    normalize_entropy = min(total_entropy / max_possible_entropy, 1.0)
    # print(f"证据 {evidence}的标准化熵: normalize_entropy={normalize_entropy}")
    return normalize_entropy

# owa相似度计算

In [ ]:
 # 字母到数字的自动映射 (A->1, B->2, ..., Z->26)
def to_numeric(x):
    if isinstance(x, str) and x.isalpha() and x in ascii_uppercase:
        return ascii_uppercase.index(x) + 1
    return int(x)  # 如果是数字字符串或数字

def compute_singleton_relation(i, j, alpha = 0.8):
    """
    通用单例元素关系计算
    支持: 字母(A-Z)、数字、或混合输入
    关系公式: 1 - exp(-|a - b|)
    """
    a = to_numeric(i)
    b = to_numeric(j)
    return alpha * (1 - np.exp(-abs(a - b))) + (1 - alpha) * np.abs(a - b)

def compute_singleton_set(A, B, gamma):
    """计算单例到集合的关系"""
    r_values = []
    for b in B:
        r_values.append(compute_singleton_relation(A, b))
    return owa_aggregation(r_values, gamma, force_binary=False)


def build_relation_matrix(focal_elements, gamma=0.8):
    """构建关系矩阵（适配字母型证据体）"""
    n = len(focal_elements)
    R_o = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):  # 仅计算上三角
            A, B = focal_elements[i], focal_elements[j]

            # Case 1: 单例-单例关系
            if len(A) == 1 and len(B) == 1:
                a = next(iter(A))  # 获取集合中的唯一元素
                b = next(iter(B))
                R_o[i, j] = compute_singleton_relation(a, b)

            # Case 2: 单例-集合关系
            elif len(A) == 1 and len(B) > 1:
                a = next(iter(A))
                R_o[i, j] = compute_singleton_set(a, B, gamma)

            # Case 3: 集合-集合关系
            elif len(A) > 1 and len(B) > 1:
                r_values = []
                for a in A:
                    r_values.append(compute_singleton_set(a, B, gamma))
                R_o[i, j] = owa_aggregation(r_values, gamma, force_binary=False)

    # 对称填充下三角
    R_o = np.triu(R_o) + np.triu(R_o, 1).T
    return R_o

def owa_aggregation(values, gamma, force_binary=False):
    """OWA聚合函数（保持不变）"""
    if len(values) == 1:
        return values[0]

    if force_binary:  # 二元情况直接计算
        sorted_v = sorted(values, reverse=True)
        return gamma * sorted_v[0] + (1 - gamma) * sorted_v[1]

    sorted_values = np.array(sorted(values, reverse=True))
    n = len(sorted_values)

    def entropy(w):
        return -np.sum(w * np.log(w + 1e-10))

    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun': lambda w: np.sum([(n - k - 1) / (n - 1) * w[k] for k in range(n)]) - gamma}
    ]
    bounds = [(0.001, 1) for _ in range(n)]
    res = minimize(entropy, x0=np.ones(n) / n, bounds=bounds,
                   constraints=constraints, options={'ftol': 1e-10})

    weights = res.x / np.sum(res.x)
    return np.dot(sorted_values, weights)

def d_OWA(m1, m2, gamma=0.8):
    """计算两个证据体之间的距离"""
    # 获取所有焦元（保持顺序一致性）
    focal_elements = sorted(list(set(m1.keys()).union(set(m2.keys()))), key=lambda x: (len(x), x))

    # 构建关系矩阵
    R_o = build_relation_matrix(focal_elements, gamma)

    # 转换为向量（保持相同顺序）
    m1_vec = np.array([m1.get(f, 0) for f in focal_elements])
    m2_vec = np.array([m2.get(f, 0) for f in focal_elements])

    # 计算距离
    diff = m1_vec - m2_vec
    M = np.eye(len(R_o)) - R_o
    distance = np.sqrt(0.5 * diff.T @ M @ diff)
    return distance

def owa_similarity_matrix(evidence, gamma=0.8):
    """计算证据体列表的相似度矩阵"""
    # 转换证据体格式
    # evidence_frozenset = [{frozenset(k): v for k, v in ev.items()} for ev in evidence]
    evidence_frozenset = evidence.copy()
    n = len(evidence_frozenset)
    similarity_matrix = np.eye(n)  # 对角线为1

    for i in range(n):
        for j in range(i + 1, n):
            distance = d_OWA(evidence_frozenset[i], evidence_frozenset[j], gamma)
            similarity = 1 - distance
            similarity_matrix[i][j] = similarity
            similarity_matrix[j][i] = similarity

    return similarity_matrix

# 置信度

In [ ]:
def compute_credibility_from_similarity(S):
    """
    计算归一化的可信度权重 Crd
    - 输入: S (相似度矩阵)
    - 输出: Crd (可信度权重向量)
    """
    # 计算支持度 Sup(m_i)
    Sup = np.sum(S, axis=1) - np.diag(S)  # 去掉自身对自身的支持度

    # 计算可信度 Crd 并归一化
    Crd = Sup / np.sum(Sup) if np.sum(Sup) != 0 else np.ones_like(Sup) / len(Sup)  # 避免除零

    return Crd

# 权重计算

In [ ]:
def compute_evidence_quality(evidence):
    """综合质量评估（RPS熵+信息量+确定性）"""
    prob = np.array(list(evidence.values()))
    prob = prob / (np.sum(prob) + 1e-10)

    # 1. RPS熵指标（取反，因为熵越低质量越高）
    rps_entropy = compute_rps_entropy(evidence)
    rps_score = 1- rps_entropy  # 已标准化

    # 2. 信息量指标（香农熵）
    # info_score = 1 - entropy(prob) / math.log(len(prob) + 1e-10)

    # 3. 确定性指标
    certainty = np.max(prob)

    # 三重加权综合（可调参数）
    return 0.7 * rps_score + 0.3 * certainty

def compute_belief_from_similarity(S, evidence_list, alpha=0.8):
    """
    增强版置信度计算（三级融合）
    参数：
    - S: 相似度矩阵
    - evidence_list: 证据体列表
    - alpha: 质量主导系数 (0-1)
    返回：
    - belief_weights: 最终权重
    """
    n = len(evidence_list)
    np.fill_diagonal(S, 0)  # 确保无自相似

    # 第一阶段：基础可信度
    Crd = compute_credibility_from_similarity(S)

    # 第二阶段：质量评估（使用改进的compute_evidence_quality）
    Quality = np.array([compute_evidence_quality(ev) for ev in evidence_list])
    Q_norm = Quality / (np.sum(Quality) + 1e-10)

    # 第三阶段：RPS熵加权一致性
    rps_weights = np.array([1 - compute_rps_entropy(ev) for ev in evidence_list])
    Consistency = np.sum(S * rps_weights.reshape(-1, 1), axis=1)
    C_norm = Consistency / (np.sum(Consistency) + 1e-10)

    # 动态融合（带可信度校准）
    # combined = (alpha * Q_norm + (1 - alpha) * C_norm) * Crd
    combined = (Q_norm + C_norm) * Crd
    belief_weights = combined / (np.sum(combined) + 1e-10)

    return belief_weights

# 软似然函数

In [ ]:
def similarity_based_soft_likelihood(omega, evidence_list, R_values, alpha):
    """
    基于相似度矩阵的软似然函数
    - 输入:
        omega: 待评估假设
        evidence_list: 证据列表（每个证据为质量函数字典）
        S: 相似度矩阵
        alpha: 乐观系数
    - 输出:
        L: 软似然值
    """
    # 获取各证据对假设的支持度
    prob_values = [ev.get(omega, 0) for ev in evidence_list]

    # 按加权支持度降序排序
    sorted_indices = np.argsort([p * R for p, R in zip(prob_values, R_values)])[::-1]

    # 计算权重向量
    w = []
    for i in range(len(evidence_list)):
        if i == 0:
            w.append(R_values[sorted_indices[i]] ** ((1 - alpha) / alpha))
        else:
            sum_k = np.sum([R_values[sorted_indices[k]] for k in range(i + 1)])
            sum_k_prev = np.sum([R_values[sorted_indices[k]] for k in range(i)])
            w.append(sum_k ** ((1 - alpha) / alpha) - sum_k_prev ** ((1 - alpha) / alpha))

    # 计算软似然值
    L = 0
    for i in range(len(evidence_list)):
        prod = np.prod([prob_values[sorted_indices[k]] for k in range(i + 1)])
        L += w[i] * prod

    return L

# 融合函数入口

In [ ]:
def fuse_evidence(evidence, R, hypotheses = {'A', 'B', 'C','D'}, alpha=0.1):
    """
    计算融合概率：
    - 使用软似然函数进行最终的概率估计
    - 输入: evidence (列表), 归一化后的证据列表；R (数组), 权重；alpha (浮点数), 乐观系数。
    - 输出: 融合后的概率分布。
    """
    # hypotheses = {
    # ('A',),
    # ('B',),
    # ('C',),
    # ('B', 'A'),
    # ('A', 'B'),
    # ('A', 'C'),
    # ('C', 'A', 'B'),
    # ('B', 'C', 'A'),
    # ('B', 'A', 'C'),
    # ('A', 'C', 'B'),
    # ('A', 'B', 'C')}
    # hypotheses = {'A', 'B', 'C','D','E'}
    fused = {}

    for omega in hypotheses:
        L = similarity_based_soft_likelihood(omega, evidence, R, alpha)
        fused[omega] = L

    # print("fused_result_before_nor:", convert_numpy_types(fused))
    # 归一化
    total_L = sum(fused.values()) + 1e-10
    fused = {k: v / total_L for k, v in fused.items()}
    print("fused_result_after_nor:", convert_numpy_types(fused))
    return fused

# OPT计算

In [ ]:
def calculate_OPT(pmf_data, hypotheses={'A', 'B', 'C','D'}):
    """
    直接基于假设集的OPT计算
    参数：
    - pmf_data: SLF融合后的概率分布字典 {tuple: prob}
    - hypotheses: 预设的假设集合
    返回：
    - 标准化后的OPT概率分布 {class: prob}
    """
    # 初始化概率分布
    P_OPT = {h: 0.0 for h in hypotheses}

    # 按命题长度排序处理（单元素优先）
    sorted_items = sorted(pmf_data.items(),
                          key=lambda x: (len(x[0]), x[0]))

    for items, mass in sorted_items:
        items_set = set(items)  # 转换为集合避免顺序影响
        if len(items_set) == 1:
            # 单元素直接分配
            theta = next(iter(items_set))  # 获取唯一元素
            P_OPT[theta] += mass
        else:
            # 多元素按OPT规则分配
            last_element = items[-1]  # 保持原顺序的最后一个元素
            for theta in items_set:
                if theta != last_element:
                    P_OPT[theta] += mass / (len(items_set) - 1)

    # 标准化处理
    total = sum(P_OPT.values())
    return {k: v / total for k, v in P_OPT.items()}

def batch_OPT(data_list, hypotheses = {'A', 'B', 'C','D'}):
    opt_list = []
    for d in data_list:
        opt_one  = calculate_OPT(d, hypotheses)
        print("opt_one:", convert_numpy_types(opt_one))
        opt_list.append(opt_one)
    return opt_list

# 主函数入口

In [ ]:
def main_function(evidence, hypotheses_all, hypotheses = {'A', 'B', 'C','D'}):
    # 处理证据体格式
    # normalized_evidence = normalize_evidence(evidence)
    # OPT_evidence = batch_OPT(evidence, hypotheses)

    # 计算相似度矩阵
    S = owa_similarity_matrix(evidence)

    # 计算权重 R
    print(S)
    # 计算综合置信度权重
    R_values = compute_belief_from_similarity(S, evidence)
    # 计算融合概率
    alpha = 0.1 # 乐观系数
    fused_probabilities = fuse_evidence(evidence, R_values, hypotheses_all, alpha)

    print("Optimal R:", R_values)
    fused_end = calculate_OPT(fused_probabilities, hypotheses)
    print("Fused probabilities (after SLF):", convert_numpy_types(fused_end))

In [ ]:
def convert_numpy_types(obj):
    """
    递归转换数据结构中的 np.int64 和 np.float64 为 Python 原生类型
    支持处理: 列表/元组/集合/字典/嵌套结构
    """
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, (str, bytes, bytearray)):
        return obj
    elif isinstance(obj, dict):
        return {convert_numpy_types(k): convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, Iterable) and not isinstance(obj, (str, bytes)):
        return type(obj)(convert_numpy_types(x) for x in obj)
    else:
        return obj

# rps生成函数

## 生成所有按顺序选择的排列组合

In [ ]:
# 生成所有按顺序选择的排列组合
def get_ordered_permutations(num_classes):
    result = []
    # 逐步增加元素数量
    for i in range(1, num_classes + 1):
        # 生成i个元素的全排列
        result.extend(itertools.permutations(range(i), i))
    return result

## 计算 weighted PMF

In [ ]:
# 计算 weighted PMF
def calculate_weighted_pmf(weight_matrix, sorted_nmv):
    num_combinations, num_attributes = weight_matrix.shape
    num_classes = sorted_nmv.shape[0]  # 获取类的数量（classes）

    # 获取排列组合
    # all_combinations = get_ordered_permutations(num_classes)

    # 初始化 weighted_pmf 矩阵
    weighted_pmf = np.zeros_like(weight_matrix)

    # 记录当前组合数对应的起始位置
    current_row = 0

    # 遍历组合大小 i（从 1 到 num_classes）
    for i in range(1, num_classes + 1):
        num_permutations = len(list(itertools.permutations(range(i), i)))  # 当前大小的排列组合数量

        # 遍历每个属性 j
        for j in range(num_attributes):
            # 对于当前大小 i 的排列组合，使用 sorted_nmv[i-1, j]
            # 组合有几个类，就乘于多少行
            weighted_pmf[current_row:current_row + num_permutations, j] = (
                    weight_matrix[current_row:current_row + num_permutations, j] * sorted_nmv[i - 1, j]
            )

        # 更新起始行
        current_row += num_permutations

    return weighted_pmf

## rps生成

In [ ]:
# 获取按顺序选择的排列组合
def gen_rps_fun(test_size=0.2):
    # 1. 加载Iris数据集并划分为训练集和测试集
    iris = load_iris()
    X = iris.data  # 四个属性
    y = iris.target  # 三个类 (0, 1, 2)
    num_classes = len(np.unique(iris.target))
    num_attributes = iris.data.shape[1]
    # 将数据集划分为训练集和测试集，乱序
    print("\n当前 (test_size):\n", test_size)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=42)
    # 2. 计算每个类中每个属性的 mean value and standard deviation (无偏估计)
    mean_std_by_class = []
    for class_label in np.unique(y_train):
        X_class = X_train[y_train == class_label]
        mean_std = [(np.mean(X_class[:, i]), np.std(X_class[:, i], ddof=1)) for i in range(X_class.shape[1])]
        mean_std_by_class.append(mean_std)

    mean_std_by_class = np.array(mean_std_by_class)
    # print("每个类中每个属性的均值和标准差:\n", mean_std_by_class)
    # print("Shape of mean_std_by_class:\n", mean_std_by_class.shape)
    # 3. 为每个类和每个属性建立高斯分布函数，并对测试集中随机选取的一个样本进行预测

    # 保存下(3,4)个Gaussian distribution函数
    # 创建一个(3,4)的函数数组，用来存储每个类中每个属性的高斯分布函数
    # 对于每个类中的每个属性，计算该样本在该属性下的高斯分布的概率密度值。
    gaussian_functions = np.empty((3, 4), dtype=object)

    # 初始化并保存高斯分布函数
    for class_label in range(num_classes):
        for i in range(num_attributes):  # 四个属性
            mean, std = mean_std_by_class[class_label, i]
            # 保存高斯分布函数
            gaussian_functions[class_label, i] = norm(loc=mean, scale=std)

    # 随机选择一个测试集中的样本
    test_sample = X_test[np.random.randint(0, len(X_test))]

    # 计算该测试样本在每个类中每个属性的高斯分布结果
    gaussian_results = []
    for class_label in range(num_classes):
        class_results = []
        for i in range(num_attributes):  # 四个属性
            # 调用保存的高斯分布函数，计算概率密度值
            # pdf是SciPy的norm对象的方法，用于计算给定值在该正态分布下的概率密度值。
            pdf_value = gaussian_functions[class_label, i].pdf(test_sample[i])
            class_results.append(pdf_value)
        gaussian_results.append(class_results)

    gaussian_results = np.array(gaussian_results)
    # print("\n测试集中选取的样本:", test_sample)
    # print("\n每个类中每个属性的高斯分布函数值:\n", gaussian_results)
    column_sums = np.sum(gaussian_results, axis=0)
    normalized_results = gaussian_results / column_sums
    # print("\n每个属性针对所有类的归一化后的MV (归一化后的高斯分布值):\n", normalized_results)
    # 对归一化后的MV（normalized membership vector）进行降序排序，并保留原始顺序的索引
    sorted_indices = np.argsort(-normalized_results, axis=0)  # 降序排序，使用负号实现降序
    sorted_nmv = np.take_along_axis(normalized_results, sorted_indices, axis=0)  # 按照索引排序后的值
    sorted_gaussian_functions = np.take_along_axis(gaussian_functions, sorted_indices, axis=0)  # 按照索引排序后的GDM

    # 打印结果
    # print("\n归一化后的MV降序排序的结果:\n", sorted_nmv)
    # print("\n每个元素排序前的原始类索引:\n", sorted_indices)
    x_mean_ord = np.empty((3, 4))
    std_ord = np.empty((3, 4))

    # mean_std_by_class 的 shape 是 (3, 4, 2)，索引 [class, attribute, 0] 获取均值，索引 [class, attribute, 1] 获取标准差
    for attr_idx in range(num_attributes):  # 对每个属性进行操作
        for class_idx in range(num_classes):  # 对每个类进行操作
            sorted_class_idx = sorted_indices[class_idx, attr_idx]  # 获取排序后的类索引
            x_mean_ord[class_idx, attr_idx] = mean_std_by_class[sorted_class_idx, attr_idx, 0]  # 获取排序后的均值
            std_ord[class_idx, attr_idx] = mean_std_by_class[sorted_class_idx, attr_idx, 1]  # 获取排序后的标准差

    # print("\n排序后的 x_mean_ord:\n", x_mean_ord)
    # print("\n排序后的 std_ord:\n", std_ord)
    supporting_degree = np.exp(-np.abs(test_sample - x_mean_ord))

    # print("\nSupporting degree (支持度):\n", supporting_degree)
    all_combinations = get_ordered_permutations(num_classes)
    print(all_combinations)
    # 初始化权重矩阵 weight_matrix
    num_combinations = len(all_combinations)  # 所有按顺序排列组合的数量 (应该是9)
    weight_matrix = np.zeros((num_combinations, num_attributes))  # (9, 4)

    # 对每个属性计算权重,也就是每个属性下的权重
    for attr_idx in range(num_attributes):
        # 取第一列，第一列对应属性1
        s = supporting_degree[:, attr_idx]  # 取出该属性对应的支持度 (3,)

        # 遍历每个组合，计算 w(i1...iu...iq)
        for comb_idx, combination in enumerate(all_combinations):
            q = len(combination)  # 该组合的长度
            weight = 1.0  # 初始化权重

            # 根据公式 (19) 计算权重
            for u in range(q):
                i_u = combination[u]  # 当前排列项 i_u
                numerator = s[i_u]  # 分子支持度
                denominator_sum = np.sum(s[list(combination[u:])])  # 分母，从 u 到 q 的支持度和
                weight *= numerator / denominator_sum  # 按公式累乘

            # 将计算好的权重保存到 weight_matrix
            weight_matrix[comb_idx, attr_idx] = weight

    # 输出权重矩阵
    # print("\n权重矩阵 (Weight matrix):\n", weight_matrix)
    weighted_pmf = calculate_weighted_pmf(weight_matrix, sorted_nmv)
    RPS_w = []
    for j in range(num_attributes):
        RPS_w_j = set()

        thetas = sorted_indices[:, j]
        weighted_pmf_j = weighted_pmf[:, j]

        for idx, combination in enumerate(all_combinations):
            A = thetas[list(combination)]
            M_A = weighted_pmf_j[idx]
            A = tuple((A))
            RPS_w_j.add((A, M_A))

        RPS_w.append(RPS_w_j)
    return RPS_w

In [ ]:
# 定义映射关系
index_to_label = {0: 'A', 1: 'B', 2: 'C'}

# 转换函数
def convert_to_labeled_rps(gen_rps):
    labeled_evidence = []
    for rps in gen_rps:
        converted = {}
        for items, mass in rps:
            # 转换数字索引为字母标签
            labeled_items = tuple(index_to_label[i] for i in items)
            converted[labeled_items] = mass
        labeled_evidence.append(converted)
    return labeled_evidence

In [ ]:
def extract_hypotheses(evidence_rps):
    """
    从RPS证据结构中抽取所有唯一的假设组合

    参数:
        evidence_rps: 包含多个RPS证据字典的列表

    返回:
        包含所有唯一假设的集合(set)，每个假设是一个元组
    """
    hypotheses = set()
    for evidence in evidence_rps:
        hypotheses.update(evidence.keys())
    return hypotheses

## RPS generation method# 方法调用

In [ ]:
     # 你的证据体
evidenceList = []
evidence5 = [
    {('A',): 0.7, ('B',): 0.15, ('C',): 0.15},
    {('B',): 0.5, ('C',): 0.5},
    {('A',): 0.7, ('B',): 0.15, ('C',): 0.15},
    {('A',): 0.7, ('B',): 0.15, ('C',): 0.15},
    {('A', 'C'): 0.8, ('B',): 0.2}
]
# evidenceList.append(evidence5)
# evidence = [
#     {('A',): 0.6, ('B',): 0.3, ('A','B'): 0.1},  # 证据体 1
#     {('A',): 0.5, ('B',): 0.4, ('A','B'): 0.1}  # 证据体 2
# ]

# evidence = [
#     {('A',): 0.5, ('B',): 0.2, ('C',): 0.3},
#     {('B',): 0.9, ('C',): 0.1},
#     {('A',): 0.55, ('B',): 0.1, ('A','C'): 0.35},
#     {('A',): 0.55, ('B',): 0.1, ('A','C'): 0.35},
#     {('A',): 0.6, ('B',): 0.1, ('A', 'C'): 0.3}
# ]

evidence1 = [
    {('A',): 0.0437, ('B',): 0.3346, ('C',): 0.2916, ('A', 'B'): 0.0437, ('A', 'C'): 0.0239, ('B', 'C'): 0.2385,
     ('A', 'B', 'C'): 0.0239},  # 证据体 1 (Sepal Length)
    {('A',): 0.0865, ('B',): 0.2879, ('C',): 0.1839, ('A', 'B'): 0.0863, ('A', 'C'): 0.0865, ('B', 'C'): 0.1825,
     ('A', 'B', 'C'): 0.0863},  # 证据体 2 (Sepal Width)
    {('A',): 1.4e-09, ('B',): 0.6570, ('C',): 0.1726, ('A', 'B'): 1.3e-09, ('A', 'C'): 1.4e-11, ('B', 'C'): 0.1704,
     ('A', 'B', 'C'): 1.4e-11},  # 证据体 3 (Petal Length)
    {('A',): 8.20e-06, ('B',): 0.6616, ('C',): 0.1692, ('A', 'B'): 8.20e-06, ('A', 'C'): 3.80e-06, ('B', 'C'): 0.1692,
     ('A', 'B', 'C'): 3.80e-06}  # 证据体 4 (Petal Width)
]
# evidenceList.append(evidence1)
# 应用一的数据
# evidence2 = [
#     {('A',): 0.40, ('B',): 0.28, ('C',): 0.30, ('A', 'C'): 0.02},  # S₁
#     {('A',): 0.01, ('B',): 0.90, ('C',): 0.08, ('A', 'C'): 0.01},  # S₂
#     {('A',): 0.63, ('B',): 0.06, ('C',): 0.01, ('A', 'C'): 0.30},  # S₃
#     {('A',): 0.60, ('B',): 0.09, ('C',): 0.01, ('A', 'C'): 0.30},  # S₄
#     {('A',): 0.60, ('B',): 0.09, ('C',): 0.01, ('A', 'C'): 0.30}   # S₅
# ]
evidence2 = [
    {('A',): 0.40, ('B',): 0.28, ('C',): 0.30, ('A', 'C'): 0.01, ('C', 'A'): 0.01},  # S₁
    {('A',): 0.01, ('B',): 0.90, ('C',): 0.08, ('A', 'C'): 0.005, ('C', 'A'): 0.005},  # S₂
    {('A',): 0.63, ('B',): 0.06, ('C',): 0.01, ('A', 'C'): 0.02, ('C', 'A'): 0.01},  # S₃
    {('A',): 0.60, ('B',): 0.09, ('C',): 0.01, ('A', 'C'): 0.02, ('C', 'A'): 0.01},  # S₄
    {('A',): 0.60, ('B',): 0.09, ('C',): 0.01, ('A', 'C'): 0.02, ('C', 'A'): 0.02}  # S₅
]
# evidenceList.append(evidence2)
#
# # 应用二的数据
evidence3 = [
    {('A',): 0.7, ('B',): 0.1, ('A', 'B', 'C'): 0.2},  # 证据体 m1
    {('A',): 0.7, ('A', 'B', 'C'): 0.3},  # 证据体 m2
    {('A',): 0.65, ('B',): 0.15, ('A', 'B', 'C'): 0.20},  # 证据体 m3
    {('A',): 0.75, ('C',): 0.05, ('A', 'B', 'C'): 0.20},  # 证据体 m4
    {('B',): 0.20, ('C',): 0.80}  # 证据体 m5
]
# evidenceList.append(evidence3)
evidence4 = [
    {('A',): 0.6, ('B',): 0.15, ('C',): 0.15, ('D',): 0, ('E',): 0.1},  # 证据体 m1
    {('A',): 0.001, ('B',): 0.45, ('C',): 0.15, ('D',): 0.24, ('E',): 0.159},  # 证据体 m2
    {('A',): 0.55, ('B',): 0.1, ('C',): 0.1, ('D',): 0.15, ('E',): 0.1},  # 证据体 m3
    {('A',): 0.8, ('B',): 0.1, ('C',): 0.05, ('D',): 0, ('E',): 0.05},  # 证据体 m3
]
# evidenceList.append(evidence4)

evidence5 = [
    {  # 传感器1
        ('A',): 0.31,
        ('B',): 0.0,
        ('C',): 0.29,
        ('A', 'C',): 0.0,
        ('C', 'A',): 0.0,
        ('A', 'B', 'C'): 0.0167,
        ('A', 'C', 'B'): 0.0167,
        ('B', 'A', 'C'): 0.0167,
        ('B', 'C', 'A'): 0.0167,
        ('C', 'A', 'B'): 0.3167,
        ('C', 'B', 'A'): 0.0167
    },
    {  # 传感器2
        ('A',): 0.0,
        ('B',): 0.8,
        ('C',): 0.2,
        ('A', 'C',): 0.0,
        ('C', 'A',): 0.0,
        ('A', 'B', 'C'): 0.0,
        ('A', 'C', 'B'): 0.0,
        ('B', 'A', 'C'): 0.0,
        ('B', 'C', 'A'): 0.0,
        ('C', 'A', 'B'): 0.0,
        ('C', 'B', 'A'): 0.0
    },
    {  # 传感器3
        ('A',): 0.27,
        ('B',): 0.07,
        ('C',): 0.21,
        ('A', 'C',): 0.0,
        ('C', 'A',): 0.0,
        ('A', 'B', 'C'): 0.025,
        ('A', 'C', 'B'): 0.025,
        ('B', 'A', 'C'): 0.025,
        ('B', 'C', 'A'): 0.025,
        ('C', 'A', 'B'): 0.325,
        ('C', 'B', 'A'): 0.025
    },
    {  # 传感器4
        ('A',): 0.25,
        ('B',): 0.05,
        ('C',): 0.3,
        ('A', 'C',): 0.09,
        ('C', 'A',): 0.31,
        ('A', 'B', 'C'): 0.0,
        ('A', 'C', 'B'): 0.0,
        ('B', 'A', 'C'): 0.0,
        ('B', 'C', 'A'): 0.0,
        ('C', 'A', 'B'): 0.3,
        ('C', 'B', 'A'): 0.0
    },
    {  # 专家
        ('A',): 0.25,
        ('B',): 0.0,
        ('C',): 0.2,
        ('A', 'C'): 0.36,
        ('C', 'A'): 0.0,
        ('A', 'B', 'C'): 0.0233,
        ('A', 'C', 'B'): 0.0733,
        ('B', 'A', 'C'): 0.0233,
        ('B', 'C', 'A'): 0.0233,
        ('C', 'A', 'B'): 0.0233,
        ('C', 'B', 'A'): 0.0233
    }
]
# evidenceList.append(evidence5)

evidence_rps = [
    {  # 第一组RPS证据
        ('A',): 0.2,
        ('B',): 0.08,
        ('C',): 0.0,
        ('B', 'A'): 0.05,
        ('A', 'B'): 0.12,
        ('A', 'C'): 0.03,
        ('C', 'A', 'B'): 0.0,
        ('B', 'C', 'A'): 0.05,
        ('B', 'A', 'C'): 0.1,
        ('A', 'C', 'B'): 0.25,
        ('A', 'B', 'C'): 0.12
    },
    {  # 第二组RPS证据
        ('A',): 0.07,
        ('B',): 0.13,
        ('C',): 0.02,
        ('B', 'A'): 0.2,
        ('A', 'B'): 0.07,
        ('A', 'C'): 0.1,
        ('C', 'A', 'B'): 0.08,
        ('B', 'C', 'A'): 0.0,
        ('B', 'A', 'C'): 0.2,
        ('A', 'C', 'B'): 0.0,
        ('A', 'B', 'C'): 0.13
    },
    {  # 第三组RPS证据
        ('A',): 0.14,
        ('B',): 0.09,
        ('C',): 0.0,
        ('B', 'A'): 0.08,
        ('A', 'B'): 0.12,
        ('A', 'C'): 0.00,
        ('C', 'A', 'B'): 0.05,
        ('B', 'C', 'A'): 0.0,
        ('B', 'A', 'C'): 0.1,
        ('A', 'C', 'B'): 0.3,
        ('A', 'B', 'C'): 0.12
    }
]
evidenceList.append(evidence_rps)
example5_2 = [
    {  # RPS₁
        ('A',): 0.0,  # 零质量可保留或移除
        ('B', 'C'): 0.05,
        ('C', 'B'): 0.05,  # 顺序敏感保留
        ('D',): 0.9
    },
    {  # RPS₂
        ('A',): 0.9,
        ('B', 'C'): 0.05,
        ('C', 'B'): 0.05,
        ('D',): 0.0
    },
    {  # RPS₃
        ('A',): 0.8,
        ('B', 'C'): 0.05,
        ('C', 'B'): 0.05,
        ('D',): 0.1
    }
]
# evidenceList.append(example5_2)
example5_3 = [
    {  # RPS₁
        ('A',): 0.9,
        ('B', 'C'): 0.01,
        ('C', 'B'): 0.0,  # 显式保留零值
        ('D',): 0.09
    },
    {  # RPS₂
        ('A',): 0.0,
        ('B', 'C'): 0.01,
        ('C', 'B'): 0.9,  # 注意与RPS₁的顺序相反
        ('D',): 0.09
    },
    {  # RPS₃
        ('A',): 0.5,
        ('B', 'C'): 0.01,
        ('C', 'B'): 0.4,  # 顺序敏感值
        ('D',): 0.09
    }
]
# evidenceList.append(example5_3)
example5_4 = [
    {  # RPS₁
        ('A', 'B', 'C'): 0.9,
        ('A', 'C', 'B'): 0.0,  # 显式保留零值
        ('B', 'A', 'C'): 0.05,
        ('D',): 0.05
    },
    {  # RPS₂
        ('A', 'B', 'C'): 0.0,
        ('A', 'C', 'B'): 0.9,  # 注意顺序差异
        ('B', 'A', 'C'): 0.05,
        ('D',): 0.05
    },
    {  # RPS₃
        ('A', 'B', 'C'): 0.5,
        ('A', 'C', 'B'): 0.4,  # 顺序敏感值
        ('B', 'A', 'C'): 0.05,
        ('D',): 0.05
    }
]
# evidenceList.append(example5_4)
# test_size_list = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]
# test_size_list = [0.3]
# for test_size in test_size_list:
#     gen_rps = gen_rps_fun(test_size)
#     print(convert_numpy_types(gen_rps))
#     labeled_evidence = convert_to_labeled_rps(gen_rps)
#     evidenceList.append(labeled_evidence)
hypotheses_all = extract_hypotheses(evidence_rps)
hypotheses = {'A', 'B', 'C','D'}
for evidence in evidenceList:
    main_function(evidence, hypotheses_all, hypotheses)